# Eigenvalues of the dielectric matrix

We compute a few eigenvalues of the dielectric matrix ($q=0$, $ω=0$) iteratively.

In [1]:
using DFTK
using Plots
using KrylovKit
using Printf

# Calculation parameters
kgrid = [1, 1, 1]
Ecut = 5

# Silicon lattice
a = 10.26
lattice = a / 2 .* [[0 1 1.]; [1 0 1.]; [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

# Compute the dielectric operator without symmetries
model = model_LDA(lattice, atoms, positions, symmetries=false)
basis = PlaneWaveBasis(model; Ecut, kgrid)
scfres = self_consistent_field(basis, tol=1e-14);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.232885011542                   -0.50    7.0
  2   -7.249828032693       -1.77       -1.39    1.0
  3   -7.251287462753       -2.84       -2.04    4.0
  4   -7.251291157997       -5.43       -2.26    5.0
  5   -7.251330243462       -4.41       -2.68    4.0
  6   -7.251335614463       -5.27       -2.91    3.0
  7   -7.251338788740       -5.50       -4.08    2.0
  8   -7.251338794929       -8.21       -4.27    3.0
  9   -7.251338798306       -8.47       -4.86    1.0
 10   -7.251338798694       -9.41       -5.47    3.0
 11   -7.251338798703      -11.04       -5.84    3.0
 12   -7.251338798704      -12.09       -6.20    3.0
 13   -7.251338798704      -12.41       -6.54    2.0
 14   -7.251338798705      -13.31       -6.84    1.0
 15   -7.251338798705      -13.75       -7.37    3.0
 16   -7.251338798705      -14.27       -7.75    2.0


Applying $ε^† ≔ (1- χ_0 K)$ …

In [2]:
function eps_fun(δρ)
    δV = apply_kernel(basis, δρ; ρ=scfres.ρ)
    χ0δV = apply_χ0(scfres, δV)
    δρ - χ0δV
end;

… eagerly diagonalizes the subspace matrix at each iteration

In [3]:
eigsolve(eps_fun, randn(size(scfres.ρ)), 5, :LM; eager=true, verbosity=3);

[ Info: Arnoldi iteration step 1: normres = 0.07425006062060636
[ Info: Arnoldi iteration step 2: normres = 0.5587193937323282
[ Info: Arnoldi iteration step 3: normres = 0.7404627901974679
[ Info: Arnoldi iteration step 4: normres = 0.2531862007167783
[ Info: Arnoldi iteration step 5: normres = 0.5878232330571194
[ Info: Arnoldi schursolve in iter 1, krylovdim = 5: 0 values converged, normres = (2.82e-02, 9.00e-02, 5.49e-01, 1.88e-01, 1.03e-03)
[ Info: Arnoldi iteration step 6: normres = 0.21590903046802679
[ Info: Arnoldi schursolve in iter 1, krylovdim = 6: 0 values converged, normres = (4.62e-03, 9.73e-02, 1.22e-01, 1.08e-01, 7.37e-02)
[ Info: Arnoldi iteration step 7: normres = 0.08916341443849894
[ Info: Arnoldi schursolve in iter 1, krylovdim = 7: 0 values converged, normres = (1.95e-04, 7.58e-03, 9.58e-03, 5.73e-02, 5.79e-02)
[ Info: Arnoldi iteration step 8: normres = 0.11388124128954534
[ Info: Arnoldi schursolve in iter 1, krylovdim = 8: 0 values converged, normres = (9.72e-